In [4]:
######################################################################################
#### [Run Imports and all cells in Data Sourcing notebook before subsequent cells]####
######################################################################################
import os
import re
import math
import numpy as np
import pandas as pd
import dask.dataframe as dd
import plotly.figure_factory as ff
import plotly
from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

def plot(figure):
    plotly.offline.iplot(figure)

## First we need to extract and organize relevant information from the ~1GB data set from the USDA Census website

In [5]:
# Read table of two-digit state FIPS prefixes

states = pd.read_csv('state_alpha_to_numeric.csv', sep=',', header=None, names=['alpha','num'])
alphas = list(states['alpha'])
nums = list(states['num'])
statenum = {state:code for (state,code) in zip(alphas,nums)}
for state in statenum.keys():
    statenum[state] = str(statenum[state]).zfill(2)

In [6]:
# Using dask to avoid loading entire data set into memory.

ag_data_dir = 'Agriculture Data'

agCensusAnimals = dd.read_csv(os.path.join(ag_data_dir, '2017_cdqt_data.txt'),
                              sep='\t', header=0,
                              dtype={'COUNTY_NAME' : 'object', 'DOMAINCAT_DESC' : 'object'})

agCensusAnimals = agCensusAnimals[
        (agCensusAnimals['CENSUS_CHAPTER'] == 2) &
        (agCensusAnimals['SECTOR_DESC'] == 'ANIMALS & PRODUCTS') &
        (agCensusAnimals['AGG_LEVEL_DESC'] == 'COUNTY') & 
        (agCensusAnimals['CENSUS_TABLE'] != 1)
    ]

agCensusAnimals = agCensusAnimals.drop(columns=['CENSUS_CHAPTER',
                              'CENSUS_TABLE',
                              'SECTOR_DESC',
                              'AGG_LEVEL_DESC'])

# Only now do we load selected rows into dataframe in memory.
agCensusAnimals = agCensusAnimals.compute()

In [7]:
# Create five digit FIPS code for referece, drop 3 and 2 digit state and county codes

state_FIPS = agCensusAnimals['STATE_FIPS_CODE'].apply(lambda x: str(int(x)).zfill(2))
county_FIPS = agCensusAnimals['COUNTY_CODE'].apply(lambda x: str(int(x)).zfill(3))
agCensusAnimals['FIPS'] = state_FIPS + county_FIPS

agCensusAnimals.drop(columns=['STATE_FIPS_CODE', 'COUNTY_CODE'], inplace=True)

In [8]:
# Break up data by state, filter out state-aggregate data

animalCensus = {
    state : agCensusAnimals[(agCensusAnimals['STATE_ALPHA'] == state) &
                            agCensusAnimals['COUNTY_NAME'].str.contains('.+', na=False)]
    for state in statenum.keys()}

In [9]:
# Let's take a look at what animal "inventory" data we have

shortdescs = list(animalCensus['TX']['SHORT_DESC'].unique())
inventory_filter = re.compile('^.* - INVENTORY')
animal_keys = list(filter(inventory_filter.match, shortdescs))
animal_keys

['CATTLE, INCL CALVES - INVENTORY',
 'CATTLE, COWS - INVENTORY',
 'CATTLE, COWS, BEEF - INVENTORY',
 'CATTLE, COWS, MILK - INVENTORY',
 'CATTLE, (EXCL COWS) - INVENTORY',
 'CATTLE, ON FEED - INVENTORY',
 'HOGS - INVENTORY',
 'SHEEP, INCL LAMBS - INVENTORY',
 'GOATS - INVENTORY',
 'GOATS, MILK - INVENTORY',
 'GOATS, ANGORA - INVENTORY',
 'GOATS, MEAT & OTHER - INVENTORY',
 'EQUINE, HORSES & PONIES - INVENTORY',
 'EQUINE, MULES & BURROS & DONKEYS - INVENTORY',
 'CHICKENS, LAYERS - INVENTORY',
 'CHICKENS, PULLETS, REPLACEMENT - INVENTORY',
 'CHICKENS, BROILERS - INVENTORY',
 'TURKEYS - INVENTORY',
 'CHUKARS - INVENTORY',
 'DUCKS - INVENTORY',
 'EMUS - INVENTORY',
 'GEESE - INVENTORY',
 'GUINEAS - INVENTORY',
 'PARTRIDGES, HUNGARIAN - INVENTORY',
 'OSTRICHES - INVENTORY',
 'PEAFOWL, HENS & COCKS - INVENTORY',
 'PHEASANTS - INVENTORY',
 'PIGEONS & SQUAB - INVENTORY',
 'QUAIL - INVENTORY',
 'RHEAS - INVENTORY',
 'CHICKENS, ROOSTERS - INVENTORY',
 'POULTRY, OTHER - INVENTORY',
 'HONEY, BEE CO

In [10]:
# Now let's break up by both state and animal

animalInventories = {
    state : 
    {key : animalCensus[state][animalCensus[state]['SHORT_DESC'] == key] for key in animal_keys}
    for state in statenum.keys()
}

# This filters out aggregate data we don't need
for state in statenum.keys():
    for key in animal_keys:
        animalInventories[state][key] = animalInventories[state][key][animalInventories[state][key]['DOMAINCAT_DESC'].isnull()]
        
# Drop rows with data omitted b/c of privacy regulations, and cast values to integers
for state in statenum.keys():
    for key in animal_keys:
        animalInventories[state][key] = animalInventories[state][key][~animalInventories[state][key]['VALUE'].str.contains('\(D\)')]
        #animalInventories[state][key]['VALUE'] = np.where(animalInventories[state][key]['VALUE'].str.contains('D'), '0', animalInventories[state][key]['VALUE'])
        animalInventories[state][key]['VALUE'] = animalInventories[state][key]['VALUE'].apply(lambda s: int(s.replace(',', '')))

# Drop unneeded columns
for state in statenum.keys():
    for key in animal_keys:
        animalInventories[state][key].drop(columns=['CENSUS_ROW', 'CENSUS_COLUMN', 'DOMAINCAT_DESC', 'STATE_ALPHA', 'STATE_NAME', 'SHORT_DESC', 'COMMODITY_DESC'], inplace=True)

In [11]:
# Set FIPS as index

for state in statenum.keys():
    for key in animal_keys:
        animalInventories[state][key].set_index('FIPS', inplace=True)

In [21]:
animalInventories.keys()

dict_keys(['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FM', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MH', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'PW', 'RI', 'SC', 'SD', 'TN', 'TX', 'UM', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'])

In [12]:
# Example table for Cattle in Washington State

animalInventories['WA']['CATTLE, INCL CALVES - INVENTORY'].head()

,COUNTY_NAME,VALUE
FIPS,,
53001,ADAMS,47301
53003,ASOTIN,9884
53005,BENTON,78863
53007,CHELAN,366
53009,CLALLAM,3435


In [19]:
import os

if not os.path.exists('inventories'):
    os.makedirs('inventories')
    
for state in animalInventories:
    if not os.path.exists(os.path.join('inventories', state)):
        os.makedirs(os.path.join('inventories', state))

for state, inventories in animalInventories.items():
    for animal, data in inventories.items():
        data.to_csv(os.path.join('inventories', state, f'{animal.split(" - ")[0]}.csv'))

## Now we load in data from four states which provide county-level data on three diseases commonly associated with animals.

In [10]:
disease_data_states = ['TX', 'NY', 'WA', 'AZ']
disease_data_diseases = ['Campylobacteriosis', 'Salmonellosis', 'STEC']
disease_data_dir = 'Disease Data'

disease_data = {
    state : {disease :
        pd.read_csv(os.path.join(disease_data_dir, state, f'{state}_{disease}.csv'),
                    header = 0,
                    sep = ',')
        for disease in disease_data_diseases}
    for state in disease_data_states}

In [11]:
def count_columns(state_alpha, disease):
    columns = list(disease_data[state_alpha][disease].columns)
    regex = re.compile(r'^\d\d\d\d COUNT$')
    return list(filter(regex.search, columns))

def rate_columns(state_alpha, disease):
    columns = list(disease_data[state_alpha][disease].columns)
    regex = re.compile(r'^\d\d\d\d RATE$')
    return list(filter(regex.search, columns))

def corresponding_rate(s):
    return s.split()[0] + ' RATE'

countyInfo = pd.read_csv('US_County_Info.csv', sep=',', header=0)
countyInfo.set_index('FIPS', inplace=True, drop=False)

def FIPS(state_alpha, county_name):
    county = re.sub(r'\s', '', county_name.upper())
    state = state_alpha
    fips = countyInfo[(countyInfo['County'].apply(lambda x: re.sub(r'\s', '', x.upper()))==county) & (countyInfo['State']==state)]['FIPS'].iloc[0]
    return str(fips)

def infer_rate(state_alpha, county_name, count, rate):
    if np.isnan(rate):
        fips = int(FIPS(state_alpha, county_name))
        pop_in_100K = float(countyInfo['Population\n(2010)'][fips]) / 100000
        return float(count) / pop_in_100K
    else:
        return float(rate)

In [12]:
# A peek at countyInfo, which has population to normalize counts by population

countyInfo.head()

,State,FIPS,County,County Seat(s),Population\n(2010),Land Area\nkm²,Land Area\nmi²,Water Area\nkm²,Water Area\nmi²,Total Area\nkm²,Total Area\nmi²,Latitude,Longitude
FIPS,,,,,,,,,,,,,
1001,AL,1001,Autauga,Prattville,54571,1539.582,594.436,25.776,9.952,1565.358,604.388,+32.536382°,–86.644490°
1003,AL,1003,Baldwin,Bay Minette,182265,4117.522,1589.784,1133.19,437.527,5250.712,2027.311,+30.659218°,–87.746067°
1005,AL,1005,Barbour,Clayton,27457,2291.819,884.876,50.865,19.639,2342.684,904.515,+31.870670°,–85.405456°
1007,AL,1007,Bibb,Centreville,22915,1612.481,622.582,9.289,3.587,1621.770,626.169,+33.015893°,–87.127148°
1009,AL,1009,Blount,Oneonta,57322,1669.962,644.776,15.157,5.852,1685.119,650.628,+33.977448°,–86.567246°


In [13]:
# Cast all data to floats, with missing values cast to NaN

for state in disease_data_states:
    for disease in disease_data_diseases:
        for col in count_columns(state, disease) + rate_columns(state, disease):
            disease_data[state][disease][col] = pd.to_numeric(disease_data[state][disease][col], errors='coerce')

In [14]:
# Example table for STEC (Shiga toxin producing e. coli) in New York State
# Notice the NaNs for rate per 100K persons for some columns!
# We'll impute missing rates fix using our county 2010 population info

disease_data['WA']['STEC'].head()

,COUNTY,2013 COUNT,2013 RATE,2014 COUNT,2014 RATE,2015 COUNT,2015 RATE,2016 COUNT,2016 RATE,2017 COUNT,2017 RATE
0,Adams,0,0.0,1,5.2,3,NaN,0,0.0,0,0.0
1,Asotin,2,NaN,1,4.6,1,NaN,0,0.0,1,NaN
2,Benton,12,6.5,9,4.9,8,4.2,12,6.3,11,5.7
3,Chelan,5,6.8,3,4.0,4,NaN,1,NaN,2,NaN
4,Clallam,2,NaN,0,0.0,2,NaN,0,0.0,4,NaN


In [15]:
# Let's impute missing rates using count and population information

for state in disease_data_states:
    for disease in disease_data_diseases:
        for count_col in count_columns(state, disease):
            rate_col = corresponding_rate(count_col)
            disease_data[state][disease][rate_col] = disease_data[state][disease].apply(lambda x: infer_rate(state, x['COUNTY'], x[count_col], x[rate_col]), axis=1)

In [16]:
# Averaging the rate and count over all available years smooths out random variation year-to-year
# This is important to do when we have small counts

for state in disease_data_states:
    for disease in disease_data_diseases:
        
        disease_data[state][disease]['AVG RATE'] = 0.0
        for column in rate_columns(state, disease):
            disease_data[state][disease]['AVG RATE'] += disease_data[state][disease][column].astype(float)
        disease_data[state][disease]['AVG RATE'] = disease_data[state][disease]['AVG RATE'] / len(rate_columns(state, disease)) 
        
        disease_data[state][disease]['AVG COUNT'] = 0.0
        for column in count_columns(state, disease):
            disease_data[state][disease]['AVG COUNT'] += disease_data[state][disease][column].astype(float)
        disease_data[state][disease]['AVG COUNT'] = disease_data[state][disease]['AVG COUNT'] / len(count_columns(state, disease))  

In [17]:
# No more NaN's, and our new average rate column!

disease_data['WA']['STEC'].head()

,COUNTY,2013 COUNT,2013 RATE,2014 COUNT,2014 RATE,2015 COUNT,2015 RATE,2016 COUNT,2016 RATE,2017 COUNT,2017 RATE,AVG RATE,AVG COUNT
0,Adams,0,0.000000,1,5.2,3,16.018795,0,0.000000,0,0.000000,4.243759,0.8
1,Asotin,2,9.249410,1,4.6,1,4.624705,0,0.000000,1,4.624705,4.619764,1.0
2,Benton,12,6.500000,9,4.9,8,4.200000,12,6.300000,11,5.700000,5.520000,10.4
3,Chelan,5,6.800000,3,4.0,4,5.520820,1,1.380205,2,2.760410,4.092287,3.0
4,Clallam,2,2.800964,0,0.0,2,2.800964,0,0.000000,4,5.601927,2.240771,1.6


In [18]:
# One last step is to re-index by FIPS code for convenience

for state in disease_data_states:
    for disease in disease_data_diseases:
        disease_data[state][disease]['FIPS'] = disease_data[state][disease]['COUNTY'].apply(lambda x: FIPS(state, x))
        disease_data[state][disease].set_index('FIPS', inplace=True)

In [19]:
disease_data['WA']['STEC'].head()

,COUNTY,2013 COUNT,2013 RATE,2014 COUNT,2014 RATE,2015 COUNT,2015 RATE,2016 COUNT,2016 RATE,2017 COUNT,2017 RATE,AVG RATE,AVG COUNT
FIPS,,,,,,,,,,,,,
53001,Adams,0,0.000000,1,5.2,3,16.018795,0,0.000000,0,0.000000,4.243759,0.8
53003,Asotin,2,9.249410,1,4.6,1,4.624705,0,0.000000,1,4.624705,4.619764,1.0
53005,Benton,12,6.500000,9,4.9,8,4.200000,12,6.300000,11,5.700000,5.520000,10.4
53007,Chelan,5,6.800000,3,4.0,4,5.520820,1,1.380205,2,2.760410,4.092287,3.0
53009,Clallam,2,2.800964,0,0.0,2,2.800964,0,0.000000,4,5.601927,2.240771,1.6


## Let's narrow down our focus to four main animals: cows, pigs, chickens, and turkeys.

In [20]:
# These three add up to the total number of cows (including calves)
cow_keys = ['CATTLE, COWS, BEEF - INVENTORY',
                 'CATTLE, COWS, MILK - INVENTORY',
                 'CATTLE, (EXCL COWS) - INVENTORY']

pig_keys = ['HOGS - INVENTORY']

# These four add up to the total number of chickens
chicken_keys = ['CHICKENS, LAYERS - INVENTORY',
                'CHICKENS, PULLETS, REPLACEMENT - INVENTORY',
                'CHICKENS, BROILERS - INVENTORY',
                'CHICKENS, ROOSTERS - INVENTORY']

turkey_keys =  ['TURKEYS - INVENTORY']

In [21]:
# Code to map increasing list of values to RGB colors
# start is a list of initial r, g, b values
# end is a list of final r, g, b values
# Color is linearly interpolated

def valsToColors(values, start, end):
    maxValue = values[-1]
    minValue = values[0]
    factors = list(map(lambda value: (value - minValue) / (maxValue - minValue), values))
    reds   = list(map(lambda t: int(round((1-t)*start[0] + t*end[0])), factors));
    greens = list(map(lambda t: int(round((1-t)*start[1] + t*end[1])), factors));
    blues  = list(map(lambda t: int(round((1-t)*start[2] + t*end[2])), factors));
    rgbs = [f'rgb({r}, {g}, {b})' for (r,g,b) in zip(reds, greens, blues)]
    return rgbs

In [28]:
def land_area(fips):
    return countyInfo.loc[int(fips)]['Land Area\nkm²']

def population(fips):
    return countyInfo.loc[int(fips)]['Population\n(2010)']

def aggregate(state_data, keys, agg_function):
    """
    agg_function should take in a fips number, as well as |keys| inputs
    (in the same order), and return a single value. Undefined inputs
    are replaced with 0.
    """
    fips = {x for key in keys for x in state_data[key].index}
    values = []
    for x in fips:
        args = [state_data[key]['VALUE'][x] if x in state_data[key]['VALUE'].index else state_data[key]['VALUE'].mean() for key in keys]
        values.append(agg_function(x, *args))
    return fips, values

animal_keys = cow_keys + pig_keys + chicken_keys + turkey_keys

fips = []
values = []
for state in disease_data_states:
    data = animalInventories[state]
    agg_function = lambda x, y1, y2, y3, y4, y5, y6, y7, y8, y9: (550*y1 + 550*y2 + 550*y3 + 180*y4 + 4*y5 + 2*y6 + 4*y7 + 4*y8 + 16*y9) / land_area(x)
    new_fips, new_values = aggregate(data, animal_keys, agg_function)
    fips += new_fips
    values += new_values

endpts = list(np.mgrid[min(values):max(values) / 4 :10j])
#endpts = list(np.mgrid[0:100:10j])

colorscale = valsToColors(endpts, [200,255,0], [255,50,0])
endpts.pop(0)
endpts.pop(-1)

fig = ff.create_choropleth(fips=fips, values=values, scope=['USA'],
                           binning_endpoints=endpts, colorscale=colorscale,
                           county_outline={'color': 'rgb(0,0,0)', 'width': 0.5},
                           legend_title='Farmed Animals per km² by kg body mass')

fig.layout.template = None
fig.update_layout(width=2500, height=1000)
fig.show()

![](Images/ag_plot.png)

In [30]:
animal_keys = cow_keys + pig_keys + chicken_keys + turkey_keys

def aggregate2(state_data, keys, agg_function):
    """
    agg_function should take in a fips number, as well as |keys| inputs
    (in the same order), and return a single value. Undefined inputs
    are replaced with 0.
    """
    fips = {x for key in keys for x in state_data[key].index}
    values = []
    for x in fips:
        args = [state_data[key]['AVG RATE'][x] if x in state_data[key]['AVG COUNT'].index else (state_data[key]['AVG COUNT']*state_data[key]['AVG COUNT']).sum() / state_data[key]['AVG COUNT'].sum() for key in keys]
        values.append(agg_function(x, *args))
    return fips, values

disease_keys = ['Campylobacteriosis', 'Salmonellosis', 'STEC']

fips = []
values = []
for state in disease_data_states:
    data = disease_data[state]
    agg_function = lambda x, y, z, w: (y + z + w)
    new_fips, new_values = aggregate2(data, disease_keys, agg_function)
    fips += new_fips
    values += new_values

endpts = list(np.mgrid[min(values):max(values):10j])
#endpts = list(np.mgrid[0:100:10j])

colorscale = valsToColors(endpts, [200,255,0], [255,50,0])
endpts.pop(0)
endpts.pop(-1)

fig = ff.create_choropleth(fips=fips, values=values, scope=['USA'],
                           binning_endpoints=endpts, colorscale=colorscale,
                           county_outline={'color': 'rgb(0,0,0)', 'width': 0.5},
                           legend_title='Incidence of STEC, Campylobacteriosis, and Salmonellosis per 100K people per km²')

fig.layout.template = None
fig.update_layout(width=2500, height=1000)
fig.show()

![](Images/animal_plot.png)